In [1]:
import ray
ray.init(num_gpus=1)

2023-07-25 17:01:03,290	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.17
Ray version:,2.6.1
Dashboard:,http://127.0.0.1:8265


In [2]:
resources = ray.cluster_resources()
print(resources)

{'object_store_memory': 1877033779.0, 'memory': 3754067559.0, 'node:__internal_head__': 1.0, 'GPU': 1.0, 'CPU': 20.0, 'node:172.21.2.236': 1.0}


In [3]:
#env = gym.make('FrankaPickPlaceFixed-v0')

In [4]:
def env_creator(env_config={}):
    import gymnasium as gym
    env = gym.make('FetchPickAndPlace-v2', max_episode_steps=100)
    #env = CustomWrapper(env)
    env.reset()
    return env

In [5]:
from ray.tune.registry import register_env
register_env("Gymnasium_Pick_Place_0", env_creator)

In [6]:
# Serialize with cloudpickle and save to a file
#with open("Gymnasium_Pick_Place_0.pkl", "wb") as f:
   # cloudpickle.dump(env_creator, f)

In [7]:
from ray import tune

In [8]:
tune.run("PPO",
             # algorithm specific configuration
             config={"env": "Gymnasium_Pick_Place_0",  #
                     "framework": "torch",
                     #"seed":958,
                     "num_gpus": 1,
                     "num_rollout_workers": 4,
                     #"num_envs_per_worker": 1,
                     #"num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 32,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2
                     },
             local_dir="/home/cocp5/Gymnasium-Robotics-RL/Gymnasium_Pick_Place_0",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"training_iteration": 50}
             )

2023-07-25 17:01:04,798	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/cocp5/anaconda3/envs/gmnsmRoboRLdev/lib/python3.9/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-07-25 17:01:06,228	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-07-25 17:01:06,292	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-07-25 17:01:06,295	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2

2023-07-25 17:01:06,394	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-25 17:01:06,395	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=112409) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=112409) 2023-07-25 17:01:08,977	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,evaluation,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_Gymnasium_Pick_Place_0_768be_00000,25600,"{'ObsPreprocessorConnector_ms': 0.019634246826171875, 'StateBufferConnector_ms': 0.0033864974975585938, 'ViewRequirementAgentConnector_ms': 0.18314647674560547}","{'num_env_steps_sampled': 25600, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 25600, 'num_agent_steps_trained': 0}",{},100,{},0,-95.14,-100,8,"{'sampler_results': {'episode_reward_max': -100.0, 'episode_reward_min': -100.0, 'episode_reward_mean': -100.0, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-100.0, -100.0], 'episode_lengths': [100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.2885952602325827, 'mean_inference_ms': 0.761358943680217, 'mean_action_processing_ms': 0.1148692417974389, 'mean_env_wait_ms': 1.20702548904808, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.01348257064819336, 'StateBufferConnector_ms': 0.003635883331298828, 'ViewRequirementAgentConnector_ms': 0.13637542724609375}}, 'episode_reward_max': -100.0, 'episode_reward_min': -100.0, 'episode_reward_mean': -100.0, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-100.0, -100.0], 'episode_lengths': [100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.2885952602325827, 'mean_inference_ms': 0.761358943680217, 'mean_action_processing_ms': 0.1148692417974389, 'mean_env_wait_ms': 1.20702548904808, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.01348257064819336, 'StateBufferConnector_ms': 0.003635883331298828, 'ViewRequirementAgentConnector_ms': 0.13637542724609375}, 'num_agent_steps_sampled_this_iter': 200, 'num_env_steps_sampled_this_iter': 200, 'timesteps_this_iter': 200, 'num_healthy_workers': 0, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}","{'learner': {'__all__': {'num_agent_steps_trained': 32.0, 'num_env_steps_trained': 512.0, 'total_loss': 0.5067082294262946}, 'default_policy': {'total_loss': 0.5067082294262946, 'policy_loss': -0.007772615063004196, 'vf_loss': 0.5152437111374155, 'vf_loss_unclipped': 10.234420160019042, 'vf_explained_var': 0.18445994472131133, 'entropy': 5.086889993399382, 'mean_kl_loss': 0.012684908430856012, 'gradients_default_optimizer_global_norm': 1.1035202768398449, 'curr_lr': 0.00015, 'curr_entropy_coeff': 0.00015, 'curr_kl_coeff': 1.3732910701946821e-05}}, 'num_env_steps_sampled': 25600, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 25600, 'num_agent_steps_trained': 0}",25600,0,25600,512,695.325,0,0,0,0,4,0,0,0,"{'cpu_util_percent': 10.15, 'ram_util_percent': 67.5}",{},{},{},"{'mean_raw_obs_processing_ms': 0.3505351547427943, 'mean_inference_ms': 0.9128635787652574, 'mean_action_processing_ms': 0.13528253290917394, 'mean_env_wait_ms': 1.3527693328534616, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 0.0, 'episode_reward_min': -100.0, 'episode_reward_mean': -95.14, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 8, 'policy_reward_min': {}, 'policy_reward_max': {}, 'po

2023-07-25 17:02:16,347	INFO tune.py:1148 -- Total run time: 71.55 seconds (69.57 seconds for the tuning loop).


2023-07-26 09:49:23,153	WARNING worker.py:2006 -- Raylet is terminated: ip=172.21.2.236, id=470f421dce5bd3e52ac71b5f4692b57365205adf365e921afb1095c7. Termination is unexpected. Possible reasons include: (1) SIGKILL by the user or system OOM killer, (2) Invalid memory access from Raylet causing SIGSEGV or SIGBUS, (3) Other termination signals. Last 20 lines of the Raylet logs:
    [state-dump] 	RuntimeEnvService.grpc_client.DeleteRuntimeEnvIfPossible - 5 total (0 active), CPU time: mean = 14.309 us, total = 71.546 us
    [state-dump] 	CoreWorkerService.grpc_client.Exit - 5 total (0 active), CPU time: mean = 12.870 us, total = 64.350 us
    [state-dump] 	RuntimeEnvService.grpc_client.GetOrCreateRuntimeEnv - 5 total (0 active), CPU time: mean = 6.661 ms, total = 33.304 ms
    [state-dump] 	Subscriber.HandlePublishedMessage_WORKER_OBJECT_LOCATIONS_CHANNEL - 4 total (0 active), CPU time: mean = 14.626 us, total = 58.505 us
    [state-dump] 	InternalPubSubGcsService.grpc_client.GcsSubscriber

In [ ]:
#import ray
import gymnasium as gym
from ray.rllib.algorithms.ppo import PPO

# Load the trained model.
#ray.init()

config = {
    "env": "Gymnasium_Pick_Place_0", 
    # add any additional configuration settings here
}
agent = PPO(config=config)
agent.restore("/home/cocp5/Gymnasium-Robotics-RL/Gymnasium_Pick_Place_0/PPO/PPO_Gymnasium_Pick_Place_0_d144a_00000_0_2023-07-25_15-52-03/checkpoint_000874")



In [ ]:
agent.evaluate()

In [ ]:
# Create the environment
env = gym.make('FetchPickAndPlace-v2', max_episode_steps=100)#(config["env"])

# Run the agent on the environment for 10 episodes.
for i_episode in range(10):
    observation, info = env.reset()
    done = False
    episode_reward = 0
    while not done:
        action = agent.compute_action(observation)
        observation, reward, terminated, truncated, info = env.step(action)
        episode_reward += reward
        env.render()
    print(f"Episode {i_episode + 1} reward: {episode_reward}")

In [ ]:
#import ray
from ray.rllib.algorithms.ppo import PPOTrainer, DEFAULT_CONFIG

# Initialize Ray.
ray.init()

# Define the configuration.
config = DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1

# Create the trainer.
trainer = PPOTrainer(config=config, env="Gymnasium_Pick_Place_0")

# Load the trained model.
trainer.restore("/path/to/the/saved/checkpoint")

# Run the agent on the environment for 10 episodes.
for i_episode in range(10):
    episode_reward = 0
    done = False
    obs = trainer.env.reset()
    while not done:
        action = trainer.compute_action(obs)
        obs, reward, done, info = trainer.env.step(action)
        episode_reward += reward
        trainer.env.render()
    print(f"Episode {i_episode + 1} reward: {episode_reward}")
